<a href="https://kritikseth.github.io/ipynbtagredirect" target="_parent"><img src="https://raw.githack.com/kritikseth/kritikseth/master/assets/icons/kritik_ipynbtagredirect.svg" alt="Kritik Seth"/></a>

In [ ]:
!pip install swachhdata -q

     |████████████████████████████████| 133kB 6.9MB/s 
     |████████████████████████████████| 102kB 5.2MB/s 
     |████████████████████████████████| 266kB 9.2MB/s 
     |████████████████████████████████| 327kB 13.4MB/s 


In [ ]:
import tensorflow as tf
from swachhdata.image import ImageNet, image_split

In [ ]:
class LeNet5:
    def __init__(self):
        self._num_classes = None
        self._epochs = None
        self._optimizer = None
        self._loss = None
        self._metrics = []
        self._X, self._y = None, None
        self.__fit, self.__train = False, False
        self._model = None
    
    def _get_architecture(self):
        return tf.keras.models.Sequential([
                    tf.keras.layers.experimental.preprocessing.Resizing(32, 32),
                    tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
                    
                    tf.keras.layers.Conv2D(filters=6, kernel_size=(5, 5), activation='tanh', strides=(1, 1), padding='same', input_shape=(32, 32, 1)),
                    tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'),

                    tf.keras.layers.Conv2D(filters=16, kernel_size=(5, 5), activation='tanh', strides=(1, 1), padding='valid'),
                    tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'),

                    tf.keras.layers.Flatten(),
                    tf.keras.layers.Dense(120, activation='tanh'),
                    tf.keras.layers.Dense(84, activation='tanh'),
                    tf.keras.layers.Dense(units=self._num_classes, activation='softmax')
        ])

    def _compile(self):
        self._model.compile(optimizer=self._optimizer,
                                           loss=self._loss,
                                           metrics=self._metrics)
    
    def fit(self, X, y, optimizer, loss, metrics=['acc'], summary=False):
        self._X, self._y = X, y
        self._optimizer, self._loss, self._metrics = optimizer, loss, metrics
        self._num_classes = len(np.unique(self._y))
        self._model = self._get_architecture()
        self._compile()
        if summary:
            print(self._model.summary())
        self.__fit = True
    
    def train(self, **kwargs):
        try:
            assert(self.__fit)
        except:
            print(f'method fit needs to be called before train')

        self._model.fit(**kwargs)
        self.__train = True
    
    def predict(self, X):
        try:
            assert(self.__train)
        except:
            print(f'model needs to be trained called before predicting')
        return self._model.predict(X)
    
    def evaluate(self, X, y):
        try:
            assert(self.__train)
        except:
            print(f'model needs to be trained called before evaluating')
        return self._model.evaluate(X, y)

In [ ]:
ch = tf.keras.callbacks.ModelCheckpoint(filepath='model.h5',
                                        monitor='val_loss',
                                        mode='auto',
                                        save_best_only=True)

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                      min_delta=0.05,
                                      patience=1,
                                      mode='min',
                                      restore_best_weights=True)

lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                          factor=0.01,
                                          patience=1,
                                          mode='min')

class MonitorAccLoss(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('val_loss') < 0.5) and (logs.get('val_acc') > 0.75):
            print('\nOptimum Loss & Accuracy achieved, cancelling training!')
            self.model.stop_training = True

mal = MonitorAccLoss()

##ImageNet Cats vs Dogs

In [ ]:
indt = {'Cat': 'n02124075', 'Dog': 'n02106662'}

img_rows, img_cols = 224, 224
input_shape = (img_rows, img_cols, 3)

num_test = num_valid = 120
test_per_cat = num_test//len(indt)
valid_per_cat = num_valid//len(indt)

input_dim = (num_test, img_rows, img_cols, 3)

BATCH_SIZE = 64

In [ ]:
imgnet = ImageNet(img_shape=input_shape, total_img=num_test*3, verbose=1)

In [ ]:
images, labels = imgnet.fetch(indt)

In [ ]:
X_train, X_test, y_train, y_test = image_split(images, labels, split_size=0.8, random_state=42)
X_test, X_val, y_test, y_val = image_split(X_test, y_test, split_size=0.5, random_state=42)

In [ ]:
lenet5 = LeNet5()

In [ ]:
lenet5.fit(X_train, y_train, optimizer='adam', loss='sparse_categorical_crossentropy')

In [ ]:
lenet5.train(x=X_train, y=y_train, validation_data=(X_val, y_val), batch_size=64, epochs=20, callbacks=[ch, es, lr, mal])

In [ ]:
lenet5.evaluate(X_test, y_test)

## CIFAR10

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [ ]:
lenet5 = LeNet5()

In [ ]:
lenet5.fit(X_train, y_train, optimizer='adam', loss='sparse_categorical_crossentropy')

In [ ]:
lenet5.train(x=X_train, y=y_train, validation_data=(X_val, y_val), batch_size=64, epochs=20, callbacks=[ch, es, lr, mal])

In [ ]:
lenet5.evaluate(X_test, y_test)

## CIFAR100

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [ ]:
lenet5 = LeNet5()

In [ ]:
lenet5.fit(X_train, y_train, optimizer='adam', loss='sparse_categorical_crossentropy')

In [ ]:
lenet5.train(x=X_train, y=y_train, validation_data=(X_val, y_val), batch_size=64, epochs=20, callbacks=[ch, es, lr, mal])

In [ ]:
lenet5.evaluate(X_test, y_test)

##MNIST

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
lenet5 = LeNet5()

In [ ]:
lenet5.fit(X_train, y_train, optimizer='adam', loss='sparse_categorical_crossentropy')

In [ ]:
lenet5.train(x=X_train, y=y_train, validation_data=(X_val, y_val), batch_size=64, epochs=20, callbacks=[ch, es, lr, mal])

In [ ]:
lenet5.evaluate(X_test, y_test)

##FashionMNIST

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
lenet5 = LeNet5()

In [ ]:
lenet5.fit(X_train, y_train, optimizer='adam', loss='sparse_categorical_crossentropy')

In [ ]:
lenet5.train(x=X_train, y=y_train, validation_data=(X_val, y_val), batch_size=64, epochs=20, callbacks=[ch, es, lr, mal])

In [ ]:
lenet5.evaluate(X_test, y_test)